In [ ]:
import os
os.path.abspath("./")

# Spacy supported functions

## Part-os-speech tags

In [ ]:
import spacy
nlp = spacy.load("en_core_sci_sm")
doc = nlp("There is no focal consolidation, pleural effusion or pneumothorax.  Bilateral nodular opacities that most likely represent nipple shadows. The cardiomediastinal silhouette is normal.  Clips project over the left lung, potentially within the breast. The imaged upper abdomen is unremarkable. Chronic deformity of the posterior left sixth and seventh ribs are noted.")
for token in doc:
    # Print the text and the predicted part-of-speech tag
    print(token.text, token.pos_)

## Syntactic dependencies

In [ ]:
import spacy
nlp = spacy.load("en_core_sci_sm")
doc = nlp("There is no focal consolidation, pleural effusion or pneumothorax.  Bilateral nodular opacities that most likely represent nipple shadows. The cardiomediastinal silhouette is normal.  Clips project over the left lung, potentially within the breast. The imaged upper abdomen is unremarkable. Chronic deformity of the posterior left sixth and seventh ribs are noted.")
for token in doc:
    print(token.text, token.pos_, token.dep_, token.head.text)

## Named entities

In [2]:
import spacy
nlp = spacy.load("en_core_sci_sm")
doc = nlp("There is no focal consolidation, pleural effusion or pneumothorax.  Bilateral nodular opacities that most likely represent nipple shadows. The cardiomediastinal silhouette is normal.  Clips project over the left lung, potentially within the breast. The imaged upper abdomen is unremarkable. Chronic deformity of the posterior left sixth and seventh ribs are noted.")
for ent in doc.ents:
    # Print the entity text and its label
    print(ent.text, ent.label_)

focal consolidation ENTITY
pleural effusion ENTITY
pneumothorax ENTITY
Bilateral ENTITY
nodular opacities ENTITY
nipple shadows ENTITY
cardiomediastinal silhouette ENTITY
Clips project ENTITY
left lung ENTITY
breast ENTITY
imaged ENTITY
upper ENTITY
abdomen ENTITY
Chronic deformity ENTITY
posterior left sixth ENTITY
ribs ENTITY


In [1]:
import scispacy
import spacy
nlp = spacy.load("en_core_sci_scibert")
doc = nlp("There is no focal consolidation, pleural effusion or pneumothorax.  Bilateral nodular opacities that most likely represent nipple shadows. The cardiomediastinal silhouette is normal.  Clips project over the left lung, potentially within the breast. The imaged upper abdomen is unremarkable. Chronic deformity of the posterior left sixth and seventh ribs are noted.")
for ent in doc.ents:
    # Print the entity text and its label
    print(ent.text, ent.label_)

pleural effusion ENTITY
pneumothorax ENTITY
Bilateral ENTITY
nodular ENTITY
nipple shadows ENTITY
cardiomediastinal silhouette ENTITY
Clips project ENTITY
left lung ENTITY
breast ENTITY
imaged ENTITY
upper abdomen ENTITY
Chronic deformity ENTITY
posterior left sixth ENTITY
ribs ENTITY


/home/yuxiangliao/anaconda3/lib/python3.9/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [3]:
import scispacy
import spacy
nlp = spacy.load("en_core_sci_lg")
doc = nlp("There is no focal consolidation, pleural effusion or pneumothorax.  Bilateral nodular opacities that most likely represent nipple shadows. The cardiomediastinal silhouette is normal.  Clips project over the left lung, potentially within the breast. The imaged upper abdomen is unremarkable. Chronic deformity of the posterior left sixth and seventh ribs are noted.")
for ent in doc.ents:
    # Print the entity text and its label
    print(ent.text, ent.label_)

focal consolidation ENTITY
pleural effusion ENTITY
pneumothorax ENTITY
Bilateral ENTITY
nodular opacities ENTITY
nipple shadows ENTITY
cardiomediastinal silhouette ENTITY
left lung ENTITY
breast ENTITY
imaged ENTITY
upper abdomen ENTITY
Chronic deformity ENTITY
posterior left sixth ENTITY
ribs ENTITY


## Pipeline

In [4]:
import scispacy
import spacy
nlp = spacy.load("en_core_sci_lg")
# 关闭词性标注器tagger和依存关系标注器parser
with nlp.select_pipes(disable=["tagger", "parser"]):
    print(nlp.pipeline)
print(nlp.pipeline)

[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec object at 0x7fd76c817b20>), ('attribute_ruler', <spacy.pipeline.attributeruler.AttributeRuler object at 0x7fd76c6c4840>), ('lemmatizer', <spacy.lang.en.lemmatizer.EnglishLemmatizer object at 0x7fd76c64ffc0>), ('ner', <spacy.pipeline.ner.EntityRecognizer object at 0x7fd76c6750b0>)]
[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec object at 0x7fd76c817b20>), ('tagger', <spacy.pipeline.tagger.Tagger object at 0x7fd76c817b80>), ('attribute_ruler', <spacy.pipeline.attributeruler.AttributeRuler object at 0x7fd76c6c4840>), ('lemmatizer', <spacy.lang.en.lemmatizer.EnglishLemmatizer object at 0x7fd76c64ffc0>), ('parser', <spacy.pipeline.dep_parser.DependencyParser object at 0x7fd76cf2af90>), ('ner', <spacy.pipeline.ner.EntityRecognizer object at 0x7fd76c6750b0>)]
